## Section 3: Impacts

## Notes on the population layer
Raster downloaded from: https://www.worldpop.org/sdi/introapi/ through the code provided in [insert link to code notebook]\
They are too heavy to be downloaded on the go by the dashboard. So, they will be resampled to a lower spatial resolution and only the population in a 500 km radius from the coastline will be kept in the final raster. Right now the spatial resolution is in the order of meters, not needed for our scope. It will be resampled to at least 10km x 10km. Each country will be saved separately. The load function will then gather the last cyclone position and only load the countries in a 500 km radius from the last cyclone position.

## Setup

In [1]:
#Necessary packages
from ipyleaflet import Choropleth, Map, basemap_to_tiles, LayersControl
import ipywidgets as widgets
from IPython.display import display
from localtileserver import get_leaflet_tile_layer, TileClient
import matplotlib
import numpy as np
import os
import pandas as pd
import rasterio
from rasterio.enums import Resampling
import requests
import json

from impacts_utils import *

In [2]:
# parameters to be imported from the main widget
initial_lat_lon = (38, -90) #first cyclone position
final_lat_lon = (60, 90) #last cyclone position

In [3]:
# variable initialization

# get mean coordinates to center the plot
mean_lat = (initial_lat_lon[0] - final_lat_lon[0])/2
mean_lon = (initial_lat_lon[1] - final_lat_lon[1])/2

coord = (mean_lat, mean_lon)

## Impacts plot

In a unique ipyleaflet, the following layers are plotted:
- Coastal Hazard
- Cyclone Hazard
- Population
- World Risk Index: from the data of the World Risk Index, retrieves and plots the expositions to coastal floodings, tsunamis and sea level rise of each country

In [4]:
# widgets for return period selection
vec_rp_coh = ["5yr", "10yr", "50yr", "100yr", "250yr", "500yr", "1000yr"]
widget_sel_rp_coh = widgets.Dropdown(options = vec_rp_coh, description = 'Return Period Coastal Hazard')
widget_sel_rp_coh.style.description_width = "180px"

vec_rp_cyh = ["50yr", "100yr", "250yr", "500yr", "1000yr"]
widget_sel_rp_cyh = widgets.Dropdown(options = vec_rp_cyh, description = 'Return Period Cyclone Hazard')
widget_sel_rp_cyh.style.description_width = "180px"

In [15]:
def update_plot(rp_coh, rp_cyh):
    #Download
    print("Dwonloading data...")
    dwnl_coastalhaz(rp_coh)
    dwnl_cyclonehaz(rp_cyh)
    dwnl_riskidx()

    #Load
    print("Loading data...")
    coh = load_coastalhaz(rp_coh)
    cyh = load_cyclonehaz(rp_cyh)

    #Plot
    m = Map(center = coord, zoom = 3)
    print("Adding Coastal Hazard")
    plot_coastalhaz(coh, rp_coh, m = m)
    print("Adding Cyclone Hazard")
    plot_cyclonehaz(cyh, rp_cyh, m = m)
    print("Adding Population")
    plot_poplayer()
    print("Adding Exposition Indexes")
    plot_riskidx(["Tsunamis", "Coastal_floods", "Sea_level_rise"], m = m)
    m.add_control(LayersControl())
    m.layout.height = "700px"

    display(m)

In [16]:
widgets.interactive(update_plot, rp_coh = widget_sel_rp_coh, rp_cyh = widget_sel_rp_cyh)

interactive(children=(Dropdown(description='Return Period Coastal Hazard', options=('5yr', '10yr', '50yr', '10…